In [7]:
#%load_ext autoreload
#%autoreload 2
from sal.utils.parser import H4ArgumentParser
from sal.config import Config
from datasets import Dataset, load_dataset
from sal.utils.data import get_dataset, save_dataset

In [2]:
print('hi 1 time')

hi 1 time


In [ ]:

load_dataset('/cluster/tufts/hugheslab/kheuto01/code/probabilistic-inference-scaling/outputs/20250613_232749_task16/', split='train')

Generating train split: 25 examples [00:00, 137.56 examples/s]


Dataset({
    features: ['problem', 'solution', 'answer', 'subject', 'level', 'unique_id', 'completions', 'scores', 'pred', 'completion_tokens'],
    num_rows: 25
})

In [ ]:
# Create Config object with specified fields
config = Config(
    dataset_start=0,           # Starting index for dataset processing
    dataset_end=25,           # Ending index for dataset processing  
    push_to_hub=True,          # Enable pushing results to HuggingFace Hub
    hub_dataset_id="your-username/your-dataset-name"  # Replace with your actual hub dataset ID
)

print("Config object created successfully!")
print(f"Dataset range: {config.dataset_start} to {config.dataset_end}")
print(f"Push to hub: {config.push_to_hub}")
print(f"Hub dataset ID: {config.hub_dataset_id}")

Config object created successfully!
Dataset range: 0 to 100
Push to hub: True
Hub dataset ID: your-username/your-dataset-name


In [10]:
# Combine all batch datasets into one JSON file
import os
import re
import json
from datasets import load_dataset, concatenate_datasets

# Base paths and configuration
outputs_base_dir = "/cluster/tufts/hugheslab/kheuto01/code/probabilistic-inference-scaling/outputs"
hub_dataset_id = "kheuton/Qwen2.5-1.5B-Instruct-bon-completions"

# Get all subdirectories in outputs
subdirs = [d for d in os.listdir(outputs_base_dir) if os.path.isdir(os.path.join(outputs_base_dir, d))]
subdirs.sort()

# Collect all datasets
all_datasets = []  # Store all datasets to combine later
task_info = []
loaded_datasets = 0

print("Loading datasets from all tasks...")
print("=" * 80)

for s, subdir in enumerate(subdirs):
    # Extract task number from directory name (format: YYYYMMDD_#####_task#)
    match = re.search(r'task(\d+)', subdir)
    if match:
        task_num = int(match.group(1))
        
        # Calculate dataset range: task 1 = 0-25, task 2 = 25-50, etc.
        dataset_start = (task_num - 1) * 25
        dataset_end = task_num * 25
        
        # Full path to the output directory
        output_dir = os.path.join(outputs_base_dir, subdir)
        
        # Load batch files for this task
        batch_count = 0
        for batch_num in range(5):  # batch_0.jsonl through batch_4.jsonl
            batch_file = os.path.join(output_dir, f"batch_{batch_num}.jsonl")
            if os.path.exists(batch_file):
                try:
                    dataset = load_dataset('json', data_files=batch_file, split='train')
                    all_datasets.append(dataset)  # Add to overall collection
                    batch_count += 1
                    loaded_datasets += 1
                    print(f"  Task {task_num:2d} batch_{batch_num}: {len(dataset)} samples")
                except Exception as e:
                    print(f"  Error loading {batch_file}: {e}")
            else:
                print(f"  Warning: {batch_file} not found")
        
        task_info.append(f"Task {task_num:2d}: [{dataset_start:3d}-{dataset_end:3d}] -> {batch_count}/5 batches loaded")

# Combine all datasets into one
print("\n" + "=" * 80)
print("COMBINING ALL DATASETS")
print("=" * 80)
print(f"Total datasets to combine: {len(all_datasets)}")

if all_datasets:
    # Concatenate all datasets
    print("Concatenating datasets...")
    combined_dataset = concatenate_datasets(all_datasets)
    print(f"Combined dataset size: {len(combined_dataset)} samples")
    
    # Create output filename based on hub dataset id
    # Convert hub_dataset_id to a safe filename
    safe_filename = hub_dataset_id.replace("/", "_").replace(":", "_")
    output_filename = f"{safe_filename}_combined.json"
    output_path = os.path.join(outputs_base_dir, output_filename)
    
    # Save as JSON
    print(f"Saving combined dataset to: {output_path}")
    
    # Convert to list of dictionaries and save as JSON
    print("Converting to JSON format...")
    data_list = [sample for sample in combined_dataset]
    
    print("Writing to file...")
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data_list, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Successfully saved {len(data_list)} samples to {output_filename}")
    print(f"📁 File location: {output_path}")
else:
    print("❌ No datasets were loaded to combine")

# Print summary
print("\n" + "=" * 80)
print("SUMMARY")
print("=" * 80)
print(f"Subdirectories processed: {len(subdirs)}")
print(f"Individual batch files loaded: {loaded_datasets}")
print(f"Tasks processed: {len(task_info)}")
print("=" * 80)
for info in task_info[:5]:  # Show first 5
    print(info)
if len(task_info) > 10:
    print("...")
    for info in task_info[-5:]:  # Show last 5
        print(info)
elif len(task_info) > 5:
    for info in task_info[5:]:  # Show remaining if <= 10 total
        print(info)
print("=" * 80)

Loading datasets from all tasks...
  Task  1 batch_0: 5 samples
  Task  1 batch_1: 5 samples


  Task  1 batch_2: 5 samples
  Task  1 batch_3: 5 samples
  Task  1 batch_4: 5 samples
  Task  2 batch_0: 5 samples
  Task  2 batch_1: 5 samples
  Task  2 batch_2: 5 samples
  Task  2 batch_3: 5 samples
  Task  2 batch_4: 5 samples
  Task  2 batch_1: 5 samples
  Task  2 batch_2: 5 samples
  Task  2 batch_3: 5 samples
  Task  2 batch_4: 5 samples
  Task  3 batch_0: 5 samples
  Task  3 batch_1: 5 samples
  Task  3 batch_2: 5 samples
  Task  3 batch_0: 5 samples
  Task  3 batch_1: 5 samples
  Task  3 batch_2: 5 samples
  Task  3 batch_3: 5 samples
  Task  3 batch_4: 5 samples
  Task  4 batch_0: 5 samples
  Task  4 batch_1: 5 samples
  Task  4 batch_2: 5 samples
  Task  3 batch_3: 5 samples
  Task  3 batch_4: 5 samples
  Task  4 batch_0: 5 samples
  Task  4 batch_1: 5 samples
  Task  4 batch_2: 5 samples
  Task  4 batch_3: 5 samples
  Task  4 batch_4: 5 samples
  Task  5 batch_0: 5 samples
  Task  5 batch_1: 5 samples
  Task  5 batch_2: 5 samples
  Task  4 batch_3: 5 samples
  Task  4 batc

In [13]:
len(subdirs)

20

In [16]:
# Simple test to check for duplicate output
print("TEST 1: This should appear once")
for i in range(3):
    print(f"TEST 2: Loop iteration {i}")
print("TEST 3: This should also appear once")

# Diagnostic test for the duplication issue
import os

print(f"Length of subdirs variable: {len(subdirs)}")
print(f"Contents of subdirs: {subdirs}")

# Test if there are duplicates in subdirs
unique_subdirs = list(set(subdirs))
print(f"Unique subdirs count: {len(unique_subdirs)}")
print(f"Are there duplicates in subdirs? {len(subdirs) != len(unique_subdirs)}")

# Simple loop test with the same subdirs variable
print("\n--- Simple loop test ---")
for i, subdir in enumerate(subdirs):
    print(f"Simple test iteration: {i}")
    if i >= 5:  # Only show first 5 to keep output manageable
        print("... (truncated)")
        break

# Test if Config creation has side effects
print("=== Testing Config creation side effects ===")

# Test creating a single Config object
print("Creating Config object...")
test_config = Config(
    dataset_start=0,
    dataset_end=25,
    push_to_hub=True,
    hub_dataset_id="test/dataset",
    output_dir="/tmp/test"
)
print("Config created successfully")

# Test a loop with Config creation
print("\n=== Testing loop with Config creation ===")
for i in range(3):
    print(f"Loop iteration {i}: before Config")
    temp_config = Config(
        dataset_start=i*25,
        dataset_end=(i+1)*25,
        push_to_hub=True,
        hub_dataset_id="test/dataset",
        output_dir=f"/tmp/test{i}"
    )
    print(f"Loop iteration {i}: after Config")

print("=== Test complete ===")

# Minimal test to reproduce the 38-times issue
# Please restart the kernel first, then run this cell

import os
outputs_base_dir = "/cluster/tufts/hugheslab/kheuto01/code/probabilistic-inference-scaling/outputs"
subdirs = [d for d in os.listdir(outputs_base_dir) if os.path.isdir(os.path.join(outputs_base_dir, d))]
subdirs.sort()

print(f"Total subdirectories found: {len(subdirs)}")

# Count how many times this prints
counter = 0
for s, subdir in enumerate(subdirs):
    print(f"iteration: {s}")
    counter += 1

print(f"Total print statements executed: {counter}")
print(f"Expected: {len(subdirs)}, Actual: {counter}")

# Test logging handlers before and after Config creation
import logging

def check_logging_handlers():
    root_logger = logging.getLogger()
    print(f"Root logger handlers: {len(root_logger.handlers)}")
    for i, handler in enumerate(root_logger.handlers):
        print(f"  Handler {i}: {type(handler).__name__} - {handler}")

print("=== BEFORE importing Config ===")
check_logging_handlers()

from sal.config import Config

print("\n=== AFTER importing Config ===")
check_logging_handlers()

print("\n=== BEFORE creating first Config object ===")
test_config1 = Config(dataset_start=0, dataset_end=25, push_to_hub=False)

print("\n=== AFTER creating first Config object ===")
check_logging_handlers()

print("\n=== BEFORE creating second Config object ===")
test_config2 = Config(dataset_start=25, dataset_end=50, push_to_hub=False)

print("\n=== AFTER creating second Config object ===")
check_logging_handlers()

# Simple test to isolate the logging issue
import logging

# Clear any existing handlers
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

print("=== Test without Config creation ===")
for i in range(3):
    print(f"iteration: {i}")

print("\n=== Test with Config creation (push_to_hub=False) ===")
from sal.config import Config

for i in range(3):
    print(f"iteration: {i}")
    # Create Config without push_to_hub to avoid network calls
    config = Config(
        dataset_start=i*25, 
        dataset_end=(i+1)*25, 
        push_to_hub=False,  # This should avoid the hub-related code
        output_dir=f"/tmp/test{i}"
    )

print(f"\nRoot logger handlers after Config creation: {len(logging.root.handlers)}")

# Test the autoreload + logging theory
import logging

print(f"Before: {len(logging.root.handlers)} handlers")

# Create first Config 
config1 = Config(dataset_start=0, dataset_end=25, push_to_hub=False)
print(f"After Config 1: {len(logging.root.handlers)} handlers")

# Create second Config
config2 = Config(dataset_start=25, dataset_end=50, push_to_hub=False) 
print(f"After Config 2: {len(logging.root.handlers)} handlers")

# Test print statements
print("\n=== Testing print with multiple handlers ===")
for i in range(3):
    print(f"iteration: {i}")
    
print(f"\nFinal handler count: {len(logging.root.handlers)}")

TEST 1: This should appear once
TEST 2: Loop iteration 0
TEST 2: Loop iteration 1
TEST 2: Loop iteration 2
TEST 3: This should also appear once
Length of subdirs variable: 20
Contents of subdirs: ['20250613_125630_task1', '20250613_132720_task2', '20250613_135801_task3', '20250613_142826_task4', '20250613_145911_task5', '20250613_164055_task6', '20250613_171253_task7', '20250613_174451_task8', '20250613_181646_task9', '20250613_184911_task10', '20250613_192138_task11', '20250613_195306_task12', '20250613_202519_task13', '20250613_205535_task14', '20250613_205731_task15', '20250613_212935_task17', '20250613_220024_task18', '20250613_223109_task19', '20250613_230202_task20', '20250613_232749_task16']
Unique subdirs count: 20
Are there duplicates in subdirs? False

--- Simple loop test ---
Simple test iteration: 0
Simple test iteration: 1
Simple test iteration: 2
Simple test iteration: 3
Simple test iteration: 4
Simple test iteration: 5
... (truncated)
=== Testing Config creation side eff